# ANÁLISE DE FUNCIONAMENTO DO MODELO RANDOM FOREST PARA ACEITE DE NAVIOS

In [102]:
#IMPORTANDO BIBLIOTECAS NECESSÁRIAS
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

## VISUALIZAÇÃO E COMPREENSÃO DOS DADOS

In [113]:
#ABRINDO ARQUIVOS EM DATAFRAMES
df_treino_teste = pd.read_csv('dados_simulacao2.csv')
df_testagem_aceitos = pd.read_csv('amostras_aceitos.csv')
df_testagem_recusados = pd.read_csv('amostras_recusados.csv')
df_testagem_mista = pd.read_csv('amostras_testagem.csv')

In [114]:
#VISUALIZAÇÃO DOS DADOS DE SIMULAÇÃO COM CLASSES 'BALANCEADAS'
df_treino_teste.head(10)

,Nome,LOA (m),Boca (m),DWT (ton),Calado de Entrada (m),Calado de Saída (m),Calado Aéreo (m),Pontal (m),Tamanho de Lança (m),Ano de Construção,Tipo do Navio,Último Porto,Próximo Porto,Situação
0,Navio1,59,15,63260,14,8,24,19,11,2008,GL,Porto5,Porto2,1
1,Navio2,15,9,46723,13,15,30,13,60,2015,GL,Porto1,Porto5,1
2,Navio3,18,11,191817,15,16,28,10,33,2009,GS,Porto3,Porto1,0
3,Navio4,18,13,195241,11,16,19,12,74,2020,GSM,Porto2,Porto5,0
4,Navio5,54,17,37299,10,15,16,14,24,2006,GLP,Porto5,Porto3,1
5,Navio6,24,15,161956,16,12,14,18,101,2019,GL,Porto1,Porto5,0
6,Navio7,34,17,69892,9,18,21,8,87,2002,GL,Porto2,Porto4,0
7,Navio8,36,6,34081,11,14,27,10,47,2000,GSM,Porto5,Porto2,0
8,Navio9,51,17,30851,16,17,10,16,62,2015,GL,Porto4,Porto5,1
9,Navio10,38,13,114217,19,14,21,15,108,2002,GSM,Porto3,Porto5,0


In [116]:
#INFOS SOBRE TIPAGEM DE DADOS DO DF
df_testagem_recusados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             60 non-null     int64 
 1   Nome                   60 non-null     object
 2   LOA (m)                60 non-null     int64 
 3   Boca (m)               60 non-null     int64 
 4   DWT (ton)              60 non-null     int64 
 5   Calado de Entrada (m)  60 non-null     int64 
 6   Calado de Saída (m)    60 non-null     int64 
 7   Calado Aéreo (m)       60 non-null     int64 
 8   Pontal (m)             60 non-null     int64 
 9   Tamanho de Lança (m)   60 non-null     int64 
 10  Ano de Construção      60 non-null     int64 
 11  Tipo do Navio          60 non-null     object
 12  Último Porto           60 non-null     object
 13  Próximo Porto          60 non-null     object
 14  Situação               60 non-null     int64 
dtypes: int64(11), object(4)
m

In [117]:
# EXCLUINDO COLUNA ERRÔNEA DOS DATAFRAMES DE TESTAGEM
df_testagem_aceitos = df_testagem_aceitos.drop(columns=['Unnamed: 0'])
df_testagem_mista = df_testagem_mista.drop(columns=['Unnamed: 0'])
df_testagem_recusados = df_testagem_recusados.drop(columns=['Unnamed: 0'])

In [118]:
df_testagem_aceitos

,Nome,LOA (m),Boca (m),DWT (ton),Calado de Entrada (m),Calado de Saída (m),Calado Aéreo (m),Pontal (m),Tamanho de Lança (m),Ano de Construção,Tipo do Navio,Último Porto,Próximo Porto,Situação
0,Navio1,59,12,93953,11,8,21,14,16,2019,GL,Porto3,Porto5,1
1,Navio3,18,7,108851,11,18,20,19,42,2018,GS,Porto1,Porto3,1
2,Navio5,54,11,47921,11,10,21,12,20,2016,GLP,Porto5,Porto4,1
3,Navio13,39,7,105647,10,18,12,8,13,2015,GLP,Porto3,Porto4,1
4,Navio15,16,8,140374,10,17,16,9,21,2014,GSM,Porto5,Porto4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Navio693,48,14,114625,14,18,19,6,70,2007,GS,Porto3,Porto2,1
296,Navio697,48,15,147175,10,13,10,6,83,2009,GLP,Porto4,Porto2,1
297,Navio700,40,12,85766,11,15,28,15,70,2018,GSM,Porto4,Porto5,1
298,Navio701,39,15,56712,15,14,13,13,23,2019,GS,Porto4,Porto3,1


In [119]:
#BALANCEAMENTO DE CLASSE DO DATAFRAME DE TREINO
df_treino_teste['Situação'].value_counts() #TREINAMENTO DO MODELO COM MAIS RECUSADOS DO QUE ACEITOS

Situação
0    592
1    408
Name: count, dtype: int64

## TRATAMENTO DOS DADOS E CONFIGURAÇÃO DO MODELO

In [120]:
#DEFINIÇÃO DOS PREVISORES E CLASSES
indices_previsores = [1, 2, 3, 4, 5, 9]

previsores_balanceados = df_treino_teste.iloc[:, indices_previsores].values
classe_balanceados = df_treino_teste.iloc[:, 13].values

previsores_desbalanceados = df_testagem_mista.iloc[:, indices_previsores].values
classe_desbalanceados = df_testagem_mista.iloc[:, 13].values

previsores_recusados = df_testagem_recusados.iloc[:, indices_previsores].values
classe_recusados = df_testagem_recusados.iloc[:, 13].values

previsores_aceitos = df_testagem_aceitos.iloc[:, indices_previsores].values
classe_aceitos = df_testagem_aceitos.iloc[:, 13].values

In [121]:
#SEPARANDO DADOS DE TESTE E TREINO
pb_train, pb_test, cb_train, cb_test = train_test_split(previsores_balanceados, classe_balanceados, train_size=0.2)
pd_train, pd_test, cd_train, cd_test = train_test_split(previsores_desbalanceados, classe_desbalanceados, train_size=0.2)
pr_train, pr_test, cr_train, cr_test = train_test_split(previsores_recusados, classe_recusados, train_size=0.2)
pa_train, pa_test, ca_train, ca_test = train_test_split(previsores_aceitos, classe_aceitos, train_size=0.2)

In [122]:
# CRIAÇÃO DO CLASSIFICADOR BASE
classificador_base = RandomForestClassifier(random_state=0)

In [123]:
# CONFIGURANDO GRIDSEARCHCV PARA MELHOR ESCOLHA DOS HIPERPARÂMETROS
grid_search = GridSearchCV(classificador_base, parametros, cv=5, scoring='recall')

In [15]:
# EXECUTANDO A BUSCA EXAUSTIVA
grid_search.fit(p_train, c_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 20, 30]},
             scoring='recall')

In [16]:
# OBTENDO MELHORES PARÂMETROS
melhores_parametros = grid_search.best_params_
melhores_parametros

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 10}

In [151]:
#{'bootstrap': False,
#'criterion': 'gini',
#'max_depth': None,
#'max_features': 'sqrt',
#'min_samples_leaf': 1,
#'min_samples_split': 2,
#@'n_estimators': 10}

#{'bootstrap': True,
#'criterion': 'gini',
#'max_depth': None,
#'max_features': 'sqrt',
#'min_samples_leaf': 1,
#'min_samples_split': 10,
#'n_estimators': 10}

In [124]:
# CLASSIFICADOR COM PARÂMETROS ADVINDOS DO GRID SEARCH
classificador = RandomForestClassifier(
    bootstrap=True,
    criterion='gini',
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=10
)

## TREINAMENTO DO MODELO COM DADOS DE SIMULAÇÃO 'BALANCEADOS'

In [125]:
classificador.fit(pb_train, cb_train)

RandomForestClassifier(n_estimators=10)

In [126]:
previsoes_balanceadas = classificador.predict(pb_test)

## AVALIAÇÃO DO MODELO COM DADOS DE SIMULAÇÃO 'BALANCEADOS'

In [127]:
# AVALIAÇÃO DO MODELO
matriz_confusao = confusion_matrix(cb_test, previsoes_balanceadas)
acuracia = accuracy_score(cb_test, previsoes_balanceadas)
precisao = precision_score(cb_test, previsoes_balanceadas)
recall = recall_score(cb_test, previsoes_balanceadas)
f1score = f1_score(cb_test, previsoes_balanceadas)

print("MATRIZ DE CONFUSÃO (VP / FP/ FN/ VN):\n", matriz_confusao)
print("\nACURÁCIA:", acuracia)
print("\nPRECISÃO:", precisao)
print("\nSENSIBILIDADE/ RECALL:", recall)
print("\nF1-SCORE:", f1score)


MATRIZ DE CONFUSÃO (VP / FP/ FN/ VN):
 [[483   0]
 [  9 308]]

ACURÁCIA: 0.98875

PRECISÃO: 1.0

SENSIBILIDADE/ RECALL: 0.9716088328075709

F1-SCORE: 0.9856


In [129]:
resultados_acuracia = cross_val_score(classificador, previsores_balanceados, classe_balanceados, cv=5, scoring='accuracy')
resultados_precisao = cross_val_score(classificador, previsores_balanceados, classe_balanceados, cv=5, scoring='precision')
resultados_recall = cross_val_score(classificador, previsores_balanceados, classe_balanceados, cv=5, scoring='recall')
resultados_f1score = cross_val_score(classificador, previsores_balanceados, classe_balanceados, cv=5, scoring='f1')

print("Resultados Acurácia: ", resultados_acuracia)
print("Acurácia média: ", np.mean(resultados_acuracia))
print("\n")

print("Resultados Precisao: ", resultados_precisao)
print("Precisao média: ", np.mean(resultados_precisao))
print("\n")

print("Resultados Recall: ", resultados_recall)
print("Precisao média: ", np.mean(resultados_recall))
print("\n")

print("Resultados f1-score: ",resultados_f1score)
print("Precisão média: ", np.mean(resultados_f1score))

Resultados Acurácia:  [1. 1. 1. 1. 1.]
Acurácia média:  1.0


Resultados Precisao:  [1. 1. 1. 1. 1.]
Precisao média:  1.0


Resultados Recall:  [1. 1. 1. 1. 1.]
Precisao média:  1.0


Resultados f1-score:  [1.         1.         0.99386503 1.         1.        ]
Precisão média:  0.9987730061349694


## AVALIAÇÃO DO MODELO COM DADOS DE SIMULAÇÃO 'DESBALANCEADOS'

In [136]:
previsoes_desbalanceadas = classificador.predict(pd_test)
previsoes_desbalanceadas

array([0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1], dtype=int64)

In [137]:
# AVALIAÇÃO DO MODELO
matriz_confusao = confusion_matrix(cd_test, previsoes_desbalanceadas)
acuracia = accuracy_score(cd_test, previsoes_desbalanceadas)
precisao = precision_score(cd_test, previsoes_desbalanceadas)
recall = recall_score(cd_test, previsoes_desbalanceadas)
f1score = f1_score(cd_test, previsoes_desbalanceadas)

print("MATRIZ DE CONFUSÃO (VP / FP/ FN/ VN):\n", matriz_confusao)
print("\nACURÁCIA:", acuracia)
print("\nPRECISÃO:", precisao)
print("\nSENSIBILIDADE/ RECALL:", recall)
print("\nF1-SCORE:", f1score)


MATRIZ DE CONFUSÃO (VP / FP/ FN/ VN):
 [[ 47   0]
 [  5 236]]

ACURÁCIA: 0.9826388888888888

PRECISÃO: 1.0

SENSIBILIDADE/ RECALL: 0.979253112033195

F1-SCORE: 0.9895178197064989
